# Shape roof workshop

## Description

The function **ggpl_build_roof** builds a roof using vertex and cell from *MKPOL*.

*input ->* HPC value (the poligonal shape of the roof)

*output ->* HPC value (the roof builded)

<img src="http://www.centralohiohomeinspector.com/Slide51.JPG">

## Variables

To complete the task, we used lots of data-structures in this workshop. We used some dictionaries and lists. In the following lines the main variables are described. 
Remember that vertex are list of three real values and cells are lists of all vertex used for each cell.

**(verts, cells, pols) -> **This tuple is the value obtained by the function *UKPOL*

**vertexDict, vertexList -> ** The first takes trace of all the vertex present in the input pol. We use the dict to merge the same vertex in one key, and use the corresponding vertex index as values. *vertexList* is simply used to iterate on merged vertex (because keys are string type).

**skel1, coveringPol -> ** These variables are used to trace the roof's bone structure and tu put the planes above it.



## Breaking down the code

Firstly the helper functions are explained, then all the code is listed and the *ggpl_build_roof* reported.

#### The result of all computation

<img src="img1.png">

### 1- Unmake the pol and create needed data structures

As said before, we use te *UKPOL* function to get the parameters. But this functions produces lots of vertices'duplicates and the decimal value are not rounded appropriately. To face this issue, two functions are used:

*round_vertex ->* Simply rounds numbers till the 3<sup>rd</sup> decimal value

	for v in verts:
		v[0]=math.fabs(round( v[0] , 3  ))
		v[1]=math.fabs(round( v[1] , 3  ))
		v[2]=math.fabs(round( v[2] , 3  ))
        
*create_dict ->* Creates the dictionary containing vertices as keys (in string type) and the list of their index in cells as values. It creates a list of all vertex too.
    
    i=1
	for v in verts:
		#creates string type keys
		key=",".join(str(x) for x in v)
		if not key in dct.keys():
			dct[key]=[i]
		else: 
			dct.get(key).append(i)
		i+=1

	for key in dct.keys():
		vertexList.append(literal_eval(key))


### 2- Replace cells vertices

Using the dictionary given by the above function *create_dict*, the function *replace_cells* looks in all the cell given by *UKPOL* and replaces the duplicates vertices'index with the index of the merged one (the dictionary's key)

    
	#iterates on each cell.
	for cell in cells:
		#iterates on tuple in the cell
		for elT in cell:
			#iterates on dictionary
			for key, vals in dct.items():
				#checks if the tuple elemente is in the list of values and if True replaces the value with the key
				if elT in vals:
					index=cell.index(elT)
					cell[index]=int(key)

#### This picture shows the polygon created using the computation of the above functions

<img src="img2.png">

### 3- Creating the planes needed for the roof

Once all the cells are replaced with the right vertices, the builder functions needs to build the appropriates planes for the roof. For sake reason's, in *put_planes*, it is decided to create only the planes which has at least one vertex with z value major than 0. 

    buildingCells=[]
	for v in vertexList:
		if v[2]> 0.0:
			i=vertexList.index(v)
			ind.append(i+1)
	
	for index in ind:
		for cell in cells:
			if index in cell and not check_y_value(cell,vertexList) and not cell in buildingCells:
				buildingCells.append(cell)
                
	return buildingCells

#### In this picture we can see that the inferior part of the roof isn't built, in according to the *put_planes* function

<img src="img4.png">

### 4- The *check_y_value* function

In the first image of this notebook lost of roof's models are shown. Some of that models are not complete, in fact they miss both the faces ortogonal to the Y axe. To let this program model all kind of roofs, we use this function. It checks if the y value of all vertex in a cell is the same. If true, as we can see from the model's picture, that face should not be created. It checks the x value too, supposing the input polygon could have a differente axes'orientation.
Follows the code of this checking function.

	check=False	
	helpList=[]
	for el in cell:
		vert=vertexList[el-1]	
		helpList.append(vert)
	temp=helpList[0]
	yValueTemp=temp[1]
	xValueTemp=temp[0]
	for v in helpList:
		if v[1]==yValueTemp or v[0]==xValueTemp:
			check=True
		else:
			return False
	return check

## Show all code used

In [3]:
from pyplasm import *
import math
from ast import literal_eval

"""round values for vertex list"""
def round_vertex(verts):
	for v in verts:
		v[0]=math.fabs(round( v[0] , 3  ))
		v[1]=math.fabs(round( v[1] , 3  ))
		v[2]=math.fabs(round( v[2] , 3  ))

"""creates a dictionary using all rounded skel's vertex. Returns the compact list of vertex too""" 
def create_dict(verts,cells):
	dct={}
	vertexList=[]
	#counter for index in vertex list
	i=1
	for v in verts:
		#creates string type keys
		key=",".join(str(x) for x in v)
		if not key in dct.keys():
			dct[key]=[i]
		else: 
			dct.get(key).append(i)
		i+=1

	for key in dct.keys():
		vertexList.append(literal_eval(key))

	return dct , vertexList
	
"""replaces the indexes of merged vertex, with the needed one"""
def replace_cell(dct,cells):
	
	#iterates on each cell.
	for cell in cells:
		#iterates on tuple in the cell
		for elT in cell:
			#iterates on dictionary
			for key, vals in dct.items():
				#checks if the tuple elemente is in the list of values and if True replaces the value with the key
				if elT in vals:
					index=cell.index(elT)
					cell[index]=int(key)
		

"""It receivs a poligon from the function produce_hpc_value() and creates the roof using vertex and cells"""
def ggpl_build_roof(val):
	roof=produce_hpc_value(val)
	skel=SKEL_2(roof)

	#gets all values from the skeleton
	(verts,cells,pol)=UKPOL(skel)
	
	#rounds the vertex values
	round_vertex(verts)

	(vertexDict,vertexList)=create_dict(verts,cells)

	#creates an helper dictionary for iterating on cells, using a counter as key
	helperDict={}
	count=1
	for el in vertexDict.keys():
		helperDict[count]=vertexDict.get(el)	
		count+=1
	
	replace_cell(helperDict,cells)
	
	finalRoof=MKPOL([vertexList,cells,None])

	

	skel1=SKEL_1(finalRoof)	
	
	#VIEW(finalRoof)
	#VIEW(skel1)
	skel1=(OFFSET([.1,.1,.3])(skel1))

	coveringCells=put_planes(vertexList,cells)
	coveringPol=MKPOL([vertexList,coveringCells,None])
	
	#VIEW(coveringPol)
	struct=STRUCT([skel1,COLOR(RED)(coveringPol)])
	VIEW(struct)


"""This function builds planes that will be put above the beams. It builds plane only for cells with at least one vertex having z!=0. 
    Moreover, according to the used models, we check if the vertex in cells have all the same x value. If true that cell isn't build"""
def put_planes(vertexList,cells):
	#list of all vertex that satisfies condition and cells needed
	ind=[]
	buildingCells=[]
	for v in vertexList:
		if v[2]> 0.0:
			i=vertexList.index(v)
			ind.append(i+1)
	
	for index in ind:
		for cell in cells:
			if index in cell and not check_y_value(cell,vertexList) and not cell in buildingCells:
				buildingCells.append(cell)
	
	return buildingCells

"""checks if all vertices in the given cell have the y value equals to 0"""
def check_y_value(cell,vertexList):
	check=False	
	helpList=[]
	for el in cell:
		vert=vertexList[el-1]	
		helpList.append(vert)
	temp=helpList[0]
	yValueTemp=temp[1]
	xValueTemp=temp[0] #if the input hpc value has a different axes'orientation
	for v in helpList:
		if v[1]==yValueTemp or v[0]==xValueTemp:
			check=True
		else:
			return False
	return check
				
"""Produces the hpc value, input for the above function"""
def produce_hpc_value(val):
	verts1=[[0,0,0],[0,6,0],[8,6,0],[8,0,0],[4,1,3],[4,5,3]]
	cells1=[[1,4,5],[2,6,3],[3,4,5,6],[1,2,6,5],[1,4,3,2]]
	pols=None
	hip_roof=MKPOL([verts1,cells1,pols])
	verts2=[[0,0,0],[2,0,3],[2,8,3],[0,8,0],[6,0,6],[6,8,6],[10,0,3],[10,8,3],[12,0,0],[12,8,0]]
	cells2=[[1,2,3,4],[2,3,6,5],[5,6,8,7],[7,8,10,9],[1,4,9,10],[1,2,5,7,9],[3,4,6,8,10]]
	gambrel_roof=MKPOL([verts2,cells2,pols])
	
	### Following this examplas,add your polygon model in this section and ###
	### update the if statement to let the program execute it              ###
	
	if val==1:
		return hip_roof
	elif val==2:
		return gambrel_roof

if __name__=='__main__':
	ggpl_build_roof(1)
	ggpl_build_roof(2)

### 5- Additional comments 

In this code two examples are made.
If you want to add your own polygon model, just create it in the *produce_hpv_value* function, following the examples.

#### The gambrel roof model

<img src="img5.png">

#### The two polygons used in input

<img src=img6.png>

<img src=img7.png>